In [ ]:
import tensorflow as tf
import os
import networks as nw

In [ ]:
# Instantiate the model
unet = nw.UNet(input_size=(256, 256, 3), num_classes=1)

out = unet.call(tf.random.normal((1, 256, 256, 3)))  # Call the model with a random input to build the model
print(out.shape) # Make sure the output shape is correct

unet.summary()

In [ ]:
def load_image(image_path):
    """Load an image, decode it, and resize to 256x256"""
    img = tf.io.read_file(image_path)               # Read image
    img = tf.image.decode_png(img, channels=3)       # Decode as PNG (assuming PNG images)
    img = tf.image.resize(img, (256, 256))           # Resize to target size
    img = img / 255.0                               # Normalize to range [0, 1]
    return img

def load_mask(mask_path):
    """Load a mask, decode it, and resize to 256x256"""
    mask = tf.io.read_file(mask_path)               # Read mask image
    mask = tf.image.decode_png(mask, channels=1)     # Decode as PNG (assuming masks are single-channel)
    mask = tf.image.resize(mask, (256, 256))         # Resize to target size
    mask = mask / 255.0                             # Normalize to range [0, 1]
    return mask

def load_data(image_folder, mask_folder):
    # Get image and mask file paths
    image_paths = [os.path.join(image_folder, fname) for fname in os.listdir(image_folder)]
    mask_paths = [os.path.join(mask_folder, fname) for fname in os.listdir(mask_folder)]
    
    # Shuffle the paths (optional)
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, mask_paths))

    # Load images and masks
    dataset = dataset.map(lambda x, y: (load_image(x), load_mask(y)))
    
    # Shuffle and batch the dataset
    dataset = dataset.shuffle(buffer_size=1000)  # Shuffle with buffer_size (adjust as needed)
    dataset = dataset.batch(batch_size=16)  # Batch size for training (adjust as needed)
    
    return dataset


In [ ]:
# Set the directory paths
image_folder = 'normalized_images'
mask_folder = 'normalized_masks'

# Load the dataset
train_dataset = load_data(image_folder, mask_folder)

# Check the dataset shapes (optional)
for images, masks in train_dataset.take(1):
    print("Image batch shape:", images.shape)
    print("Mask batch shape:", masks.shape)

In [ ]:
# Determine the number of GPUs available
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print(tf.config.list_physical_devices('GPU'))

print(tf.__version__)


print("Is TensorFlow using GPU? ", tf.test.is_gpu_available())
print("GPU Details: ", tf.config.experimental.list_physical_devices('GPU'))


In [ ]:
# Compile the model
unet.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
unet.fit(train_dataset, epochs=5)  # Adjust the number of epochs as needed

# Save the model
unet.save('unet_model.h5')  # Save the model in .h5 format

In [ ]:
# Load the model
loaded_model = tf.keras.models.load_model('unet_model.h5')

# Evaluate the model
loss, accuracy = loaded_model.evaluate(test_dataset)

print("Restored model, accuracy: {:5.2f}%".format(100*accuracy))

# Predict on a sample image
sample_image = load_image('normalized_images/test.png')
sample_image = tf.expand_dims(sample_image, 0)  # Add batch dimension
prediction = loaded_model.predict(sample_image)


